In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

df = pd.read_csv('data/train.csv')
#df.shape

pixels_col = [col for col in df.columns if col != 'label']
samples = df[pixels_col].values


In [2]:
def one_hot_digits(x):
    res = np.zeros((10,), dtype=float)
    res[x] = 1.0
    return(res)
target = np.apply_along_axis(one_hot_digits, axis=1, arr=df[['label']])
target.shape

(42000, 10)

In [3]:
from sklearn.model_selection import train_test_split
train_samples, X_test, train_target, y_test = train_test_split(samples, target, test_size=0.2, random_state=4242)
train_target.shape

(33600, 10)

In [4]:
batch_size = 100
X = tf.placeholder(tf.float32, shape=(None, 784))
y = tf.placeholder(tf.float32)


<h3>Deep Neural Network Model:</h3>

<p>layers weigh shape :  (N_samples X N_features)(input of the layer) * (weights of the layer)(N_features X N_outputs) => (outputof the layer)((N_samples X N_outputs) + biases(N_outputs X 1)) </p>
<p>biases shape: N_outputs X 1 </p>

In [5]:
hl1_no_nodes = 500

hl2_no_nodes = 500

hl3_no_nodes = 500

n_classes = 10


hidden_layer_one = {'weights': tf.Variable(tf.random_normal(shape = [784, hl1_no_nodes])),
                    'biases': tf.Variable(tf.random_normal( shape = [hl1_no_nodes]))}
    
hidden_layer_two = {'weights': tf.Variable(tf.random_normal(shape = (hl1_no_nodes, hl2_no_nodes))),
                    'biases': tf.Variable(tf.random_normal( shape = [hl2_no_nodes]))}
    
hidden_layer_three = {'weights': tf.Variable(tf.random_normal(shape = [hl2_no_nodes, hl3_no_nodes])),
                      'biases': tf.Variable(tf.random_normal( shape = [hl3_no_nodes]))}
    
output_layer = {'weights': tf.Variable(tf.random_normal(shape = [hl3_no_nodes, n_classes])),
                'biases': tf.Variable(tf.random_normal( shape = [n_classes]))}
    


<p> (outputof the layer) = ((N_samples X N_outputs) + biases(N_outputs X 1))</p>
<p>we apply rectifier leanier unit (relu) which is an activation function and is similar to sigomoid function</p>

In [6]:
layer1_output = tf.add(tf.matmul(X, hidden_layer_one['weights']) , hidden_layer_one['biases'])
layer1_output = tf.nn.relu(layer1_output)

layer2_output = tf.add(tf.matmul(layer1_output , hidden_layer_two['weights']) , hidden_layer_two['biases'])
layer2_output = tf.nn.relu(layer2_output)

layer3_output = tf.add(tf.matmul(layer2_output, hidden_layer_three['weights']) , hidden_layer_three['biases'])
layer3_output = tf.nn.relu(layer3_output)

output = tf.matmul(layer3_output, output_layer['weights']) + output_layer['biases']




<h3>Training the neural network:</h3>

<p>softmax is a logistic function () which predcits the probability of the classes (similar to sigmoid function)</p>
<p>cross entropy: calcualtes the error or the difference between the predition and the target</p>
<p>and reduce_mean (without specifiying the axis of cal) it returns calculates the mean of a tensor elements and  a single value</p>
<p>and adamoptimizer minimizes the cost function</p>

In [7]:
prediction = output

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))

optimizer = tf.train.AdamOptimizer().minimize(cost)


</p>we extecute opration objects by running session: </p>

<p>prediction or output is tensor of probablity, for finding the accryacy 
we can find the index of max value of probablity for each sample by using argmax and which gives
us the digit and compare it to the target value of the sample:</p>

<p> we use 100 cycle of feedforward to train the samples: epoch=100</p>

In [12]:
from sklearn.utils import shuffle

epochs = 100
save_path = "/home/mahsarm/Desktop/ML-projects/NN-digits/tf_3nn_model/"
#saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(1, epochs+1):
        epoch_loss = 0
        
        X_train, y_train = shuffle(train_samples, train_target, random_state=45)
        for _ in range(train_samples.shape[0]/batch_size):
           
            train_feed_dict = {X: X_train[(_*batch_size):((_+1)*batch_size)],
                         y:y_train[(_*batch_size):((_+1)*batch_size)]}
            _, c = sess.run([optimizer, cost], train_feed_dict)
            #print(c)
            epoch_loss += c
        print('Epoch', epoch, 'completed out of', epochs, 'loss:', epoch_loss)
    #saving all the variables in the sess:
    #saver.save(sess, save_path = save_path, global_step=0)
    test_feed_dict = {X: X_test, y:y_test}
    is_correct = tf.equal(tf.argmax(prediction,1) , tf.argmax(y,1))    
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('Accuracy:', sess.run(accuracy*100, feed_dict=test_feed_dict))

        
        
       




('Epoch', 1, 'completed out of', 100, 'loss:', 406771082.640625)
('Epoch', 2, 'completed out of', 100, 'loss:', 86132229.869140625)
('Epoch', 3, 'completed out of', 100, 'loss:', 46325805.282958984)
('Epoch', 4, 'completed out of', 100, 'loss:', 27434520.31918335)
('Epoch', 5, 'completed out of', 100, 'loss:', 16944088.495727539)
('Epoch', 6, 'completed out of', 100, 'loss:', 10544153.481281281)
('Epoch', 7, 'completed out of', 100, 'loss:', 7163444.1499710083)
('Epoch', 8, 'completed out of', 100, 'loss:', 5013385.278137207)
('Epoch', 9, 'completed out of', 100, 'loss:', 3415775.8520736694)
('Epoch', 10, 'completed out of', 100, 'loss:', 2996060.2066385746)
('Epoch', 11, 'completed out of', 100, 'loss:', 2707861.229552269)
('Epoch', 12, 'completed out of', 100, 'loss:', 2546843.9262390137)
('Epoch', 13, 'completed out of', 100, 'loss:', 2045772.6559867859)
('Epoch', 14, 'completed out of', 100, 'loss:', 2025723.8417015076)
('Epoch', 15, 'completed out of', 100, 'loss:', 2494893.024621